# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [4]:
import requests

In [5]:
client_id = '5senwnEqxeH82m5OZ1q-NQ'
api_key = '-nB2cny8cbRJi9JOddCcuIvQ26lCzDWo4EVhEqIZlQRqyFGicIT8p6ORE-KuDYs6uQJ3pX0ybaA9da-F_8Q_E2EZ63j_IDfdBUHtYnlyZila6D_taEuHBWnIyzISXXYx'

In [6]:
url = 'https://api.yelp.com/v3/businesses/search'
response = requests.get(url)

ModuleNotFoundError: No module named 'yelp'

NameError: name 'Client' is not defined

In [12]:
term = 'Sushi' # 
location = 'Williamsburg' # 
SEARCH_LIMIT = 10

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(MY_API_KEY),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }
response = requests.get(url, headers=headers, params=url_params)
print(response) # 200
print(type(response.text)) # string jason format from here you have format using json
print(response.text) # term these into a dataframe

<Response [200]>
<class 'str'>
{"businesses": [{"id": "lxQ-tzFmL1tmUqZUN-CSRQ", "alias": "bozu-brooklyn", "name": "Bozu", "image_url": "https://s3-media3.fl.yelpcdn.com/bphoto/2gAySXaCsEOyXQ4YfpR35w/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/bozu-brooklyn?adjust_creative=QWmKp6NvwkIFdkx9db8P8Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QWmKp6NvwkIFdkx9db8P8Q", "review_count": 798, "categories": [{"alias": "japanese", "title": "Japanese"}, {"alias": "asianfusion", "title": "Asian Fusion"}], "rating": 4.0, "coordinates": {"latitude": 40.7129, "longitude": -73.95749}, "transactions": [], "price": "$$", "location": {"address1": "296 Grand St", "address2": "", "address3": "", "city": "Brooklyn", "zip_code": "11211", "country": "US", "state": "NY", "display_address": ["296 Grand St", "Brooklyn, NY 11211"]}, "phone": "+17183847770", "display_phone": "(718) 384-7770", "distance": 262.3456087628708}, {"id": "T2fE7hGS83Ba-QNCOqbK4A", "alias": "ako-bro

In [15]:
import json

In [16]:

shusi_yelp = json.loads(response.text)
shusi_yelp['businesses']

[{'id': 'lxQ-tzFmL1tmUqZUN-CSRQ',
  'alias': 'bozu-brooklyn',
  'name': 'Bozu',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/2gAySXaCsEOyXQ4YfpR35w/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/bozu-brooklyn?adjust_creative=QWmKp6NvwkIFdkx9db8P8Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QWmKp6NvwkIFdkx9db8P8Q',
  'review_count': 798,
  'categories': [{'alias': 'japanese', 'title': 'Japanese'},
   {'alias': 'asianfusion', 'title': 'Asian Fusion'}],
  'rating': 4.0,
  'coordinates': {'latitude': 40.7129, 'longitude': -73.95749},
  'transactions': [],
  'price': '$$',
  'location': {'address1': '296 Grand St',
   'address2': '',
   'address3': '',
   'city': 'Brooklyn',
   'zip_code': '11211',
   'country': 'US',
   'state': 'NY',
   'display_address': ['296 Grand St', 'Brooklyn, NY 11211']},
  'phone': '+17183847770',
  'display_phone': '(718) 384-7770',
  'distance': 262.3456087628708},
 {'id': 'T2fE7hGS83Ba-QNCOqbK4A',
  'alias

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [26]:
# Your code here; use a functio
response.json().keys()

shusi_yelp['total']
shusi_yelp['businesses'][0]
#n or loop to retrieve all the results from your original request

{'id': 'lxQ-tzFmL1tmUqZUN-CSRQ',
 'alias': 'bozu-brooklyn',
 'name': 'Bozu',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/2gAySXaCsEOyXQ4YfpR35w/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/bozu-brooklyn?adjust_creative=QWmKp6NvwkIFdkx9db8P8Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=QWmKp6NvwkIFdkx9db8P8Q',
 'review_count': 798,
 'categories': [{'alias': 'japanese', 'title': 'Japanese'},
  {'alias': 'asianfusion', 'title': 'Asian Fusion'}],
 'rating': 4.0,
 'coordinates': {'latitude': 40.7129, 'longitude': -73.95749},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '296 Grand St',
  'address2': '',
  'address3': '',
  'city': 'Brooklyn',
  'zip_code': '11211',
  'country': 'US',
  'state': 'NY',
  'display_address': ['296 Grand St', 'Brooklyn, NY 11211']},
 'phone': '+17183847770',
 'display_phone': '(718) 384-7770',
 'distance': 262.3456087628708}

In [23]:
for item in shusi_yelp.keys():
    print(item)

businesses
total
region


In [30]:
for k , v in shusi_yelp.items():
    for i in k:
        print(i)

b
u
s
i
n
e
s
s
e
s
t
o
t
a
l
r
e
g
i
o
n


## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [ ]:
#Your code here

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
#Your code here

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!